## Scale Invariant Feature Transform - SIFT
***

Embora os recursos de canto sejam "interessantes", eles não são bons o suficiente para caracterizar as partes verdadeiramente interessantes. Quando falamos de análise de conteúdo de imagem, queremos que a assinatura da imagem seja invariante para coisas como escala, rotação,  iluminação  e  assim  por  diante.  Os  seres  humanos  são  muito  bons  nessas  coisas. Mesmo que eu mostre uma imagem de uma maçã de cabeça para baixo que esteja esmaecida, você  ainda  vai  reconhecer a  maça.  Se  eu  mostrar  uma  versão  realmente  ampliada  dessa imagem, você ainda irá reconhecê-la. Queremos que nossos sistemas de reconhecimento de imagens sejam capazes de fazer o mesmo.

Consideremos os recursos de canto. Se você ampliar uma imagem, um canto pode deixar de ser um canto como mostrado abaixo.

![img](https://user-images.githubusercontent.com/14116020/65010782-51a00f80-d8e7-11e9-8274-2251ae746053.png)

No segundo caso, o detector não vai pegar este canto. E, uma vez que foi retirada na imagem original, a segunda imagem não será igualada com a primeira. É basicamente a mesma imagem, mas o método baseado em recursos de canto perderá totalmente o keypoint. Isso significa  que  o  detector  de  canto  não  é  exatamente  invariante  em  escala.  É  por  isso  que precisamos de um método melhor para caracterizar uma imagem.

O SIFT é um dos algoritmos mais populares em toda a visão computacional. Você pode ler o documento original de David Lowe em http://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf.

Podemos  usar  esse  algoritmo  para  extrairpontos-chave  e  criar  os  descritores  de recursos  correspondentes.  Para  identificar  um  ponto  chave  potencial,  o  SIFT  constrói  uma pirâmide ao diminuir a imagem e calcular a diferença gaussiana. Isso significa que executamos um filtro gaussiano em cada nível e calculamos a diferença para construir os níveis sucessivos na pirâmide. Para ver se o ponto atual é um ponto chave, ele analisa os vizinhos, bem como os pixels na mesma localização em níveis vizinhos da pirâmide. Se é um máximo, então o ponto atual é apanhado como um ponto chave. Isso garante que nós mantivemos a escala de chave invariante.

![img](https://user-images.githubusercontent.com/14116020/65010860-9fb51300-d8e7-11e9-8ef2-a7627b450fa6.png)

Agora que sabemos como ele calcula a invariância de escala, vejamos como ele alcança a  invariância  de  rotação.  Uma  vez  que  identificamos  os  pontos-chave,  cada  ponto  chave  é atribuído  a  uma  orientação.  Nós obtemos  a  vizinhança em  torno  de  cada  ponto  chave  e calculamos a magnitude e a direção do gradiente. Isso nos dá uma sensação da direção desse ponto  chave.  Se  tivermos  essa  informação,  poderemos  combinar  esse  ponto-chave  com  o mesmo  ponto  em  outra  imagem,  mesmo  que  seja  girado.  Uma  vez  que  conhecemos a orientação, poderemos normalizar esses pontos-chave antes de fazer as comparações.

Uma vez que temos toda essa informação, como podemos quantificá-la? Precisamos convertê-la em  um  conjunto  de  números  para  que  possamos  fazer  algum  tipo  de correspondência. Para conseguir isso, nós apenas obtemos a vizinhança 16x16 em torno de cada ponto chave e dividimos em 16 blocos de tamanho 4x4. Para cada bloco, calculamos o histograma de orientação com 8 compartimentos. Então, temos um vetor de comprimento 8 associado  a cada  bloco,  o  que  significa  que  a  vizinhança  é  representadapor  um  vetor  de tamanho 128 (8x16). Este é o descritor que será usado. Se extraímos N pontos-chave de uma imagem,  teremos  N  descritores  de  comprimento  128  cada.  Esta  matriz  de  N  descritores caracteriza a imagem dada. Considere a seguinte imagem:

![img](https://user-images.githubusercontent.com/14116020/65010926-f4f12480-d8e7-11e9-806e-eaa367c571c1.png)

Se você extrair os locais do ponto-chave usando o SIFT, você verá algo como a imagem abaixo, onde o tamanho do círculo indica a força dos pontos-chave e a linha dentro do círculo indica a orientação:

![img](https://user-images.githubusercontent.com/14116020/65010964-19e59780-d8e8-11e9-84f7-c3ec6029a6b0.png)

**Obs**:  O  SIFT não  pode  ser  usado  livremente  para  fins  comerciais  e  não  está presente  na instalação padrão do OpenCV, sendo necessário a instalação do pacote opencv-contrib com módulos  extras. Seu  uso  para  fins  comerciais  requer  autorização  dos  mantenedores  do algoritmo.

***

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2


def show(img, title=""):
    """
    Redimensionar a imagem e mostrar no codigo.
    """

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.rcParams['figure.figsize'] = (50, 50)
    plt.title(title, fontdict={"fontsize": 100})
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
# Carregando a imagem e convertendo para Grayscale
input_image = cv2.imread('imagens/plataforma.png')
gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Criando o detector de recursos SIFT
sift = cv2.xfeatures2d.SIFT_create()
keypoints = sift.detect(gray_image, None)

In [ ]:
# Desenhando os keypoints na imagem
input_image = cv2.drawKeypoints(
    input_image, keypoints, gray_image,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
# Mostra a forma dos pontos-chave e a matriz de descritores invariantes locais
kp, des = sift.detectAndCompute(gray_image,None)
print("Número de keypoints Detectados: {}".format(len(kp)))
print("Shape do Vetor de Recursos: {}".format(des.shape))

Número de keypoints Detectados: 975

Shape do Vetor de Recursos: (975, 128)

In [ ]:
# Print
show(input_image, 'SIFT Features')

![img](https://user-images.githubusercontent.com/14116020/65010964-19e59780-d8e8-11e9-84f7-c3ec6029a6b0.png)